<a href="https://colab.research.google.com/github/rj7shakya/mlprojects/blob/master/Taxi_Demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls '/content/drive/My Drive/ml_data'

yellow_tripdata_2020-01.csv
